In [24]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
import torch
from transformers import BertTokenizer, BertModel

# Download required NLTK packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
# Function to preprocess text
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'\d+%', '', text)  # Remove percentages
    text = re.sub(r'[^\w\s]', ' ', text)  # Replace punctuation with space
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

In [26]:
# Function to calculate cosine similarity between two texts
def calculate_cosine_similarity(text1, text2):
    if pd.isna(text1) or pd.isna(text2) or text1 == "" or text2 == "":
        return 0.0
    
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform([text1, text2])
        return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    except:
        return 0.0

# Function to calculate BLEU score between two texts
def calculate_bleu_score(reference_text, candidate_text):
    if pd.isna(reference_text) or pd.isna(candidate_text) or reference_text == "" or candidate_text == "":
        return 0.0
    
    # Tokenize texts
    reference_tokens = nltk.word_tokenize(reference_text.lower())
    candidate_tokens = nltk.word_tokenize(candidate_text.lower())
    
    # Apply smoothing function for short texts
    smoothie = SmoothingFunction().method1
    
    try:
        # Calculate BLEU score with different n-gram weights
        bleu1 = sentence_bleu([reference_tokens], candidate_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
        bleu2 = sentence_bleu([reference_tokens], candidate_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
        bleu3 = sentence_bleu([reference_tokens], candidate_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
        bleu4 = sentence_bleu([reference_tokens], candidate_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
        
        # Average BLEU scores
        avg_bleu = (bleu1 + bleu2 + bleu3 + bleu4) / 4
        return avg_bleu
    except:
        return 0.0

# Function to calculate METEOR score between two texts
def calculate_meteor_score(reference_text, candidate_text):
    if pd.isna(reference_text) or pd.isna(candidate_text) or reference_text == "" or candidate_text == "":
        return 0.0
    
    # Tokenize texts
    reference_tokens = nltk.word_tokenize(reference_text.lower())
    candidate_tokens = nltk.word_tokenize(candidate_text.lower())
    
    try:
        return meteor_score([reference_tokens], candidate_tokens)
    except:
        return 0.0

# Function to calculate BERT score
def calculate_bert_score(reference_text, candidate_text):
    if pd.isna(reference_text) or pd.isna(candidate_text) or reference_text == "" or candidate_text == "":
        return 0.0
    
    # Load pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    # Tokenize and encode texts
    inputs1 = tokenizer(reference_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(candidate_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Move inputs to the same device as model
    inputs1 = {k: v.to(device) for k, v in inputs1.items()}
    inputs2 = {k: v.to(device) for k, v in inputs2.items()}
    
    # Get embeddings
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    
    # Use CLS token embeddings for sentence representation
    embeddings1 = outputs1.last_hidden_state[:, 0, :].cpu().numpy()
    embeddings2 = outputs2.last_hidden_state[:, 0, :].cpu().numpy()
    
    # Calculate cosine similarity between embeddings
    similarity = cosine_similarity(embeddings1, embeddings2)[0][0]
    return similarity

In [27]:
# Improved function to standardize disease names
def standardize_disease_name(name):
    if pd.isna(name):
        return ""
    
    name = str(name).lower().strip()
    # Remove percentages
    name = re.sub(r'\d+%', '', name).strip()
    
    # Common variations of disease names to standardize - EXPANDED
    mapping = {
        # Demam (Fever) related
        'dbd': 'demam berdarah dengue',
        'dengue fever': 'demam berdarah dengue',
        'demam dengue': 'demam berdarah dengue',
        'dengue': 'demam berdarah dengue',
        'demam berdarah': 'demam berdarah dengue',
        'demam biasa': 'common fever',
        'common fever': 'common fever',
        'demam umum': 'common fever',
        
        # Gastro related
        'gastroenteritis akut': 'gastroenteritis',
        'gastroenteritis (ge) akut': 'gastroenteritis',
        'ge akut': 'gastroenteritis',
        'gastroenteritis': 'gastroenteritis',
        'diare akut': 'gastroenteritis',
        
        # Respiratory related
        'infeksi saluran pernapasan atas': 'ispa',
        'ispa': 'ispa',
        'infeksi saluran napas atas': 'ispa',
        'infeksi saluran pernafasan atas': 'ispa',
        'infeksi saluran pernapasan': 'ispa',
        
        # GERD related
        'reflux gastroesofagus': 'gerd',
        'reflux asam lambung': 'gerd',
        'refleks asam lambung': 'gerd',
        'gastroesophageal reflux disease': 'gerd',
        'refleks gastroesofagus': 'gerd',
        'gerd': 'gerd',
        
        # Gastritis related
        'maag': 'gastritis',
        'gastritis akut': 'gastritis',
        'penyakit maag': 'gastritis',
        'penyakit maag akut': 'gastritis',
        'gastritis': 'gastritis',
        
        # Heart related
        'infark miokard akut': 'serangan jantung',
        'serangan jantung': 'serangan jantung',
        'angina pektoris': 'angina',
        'angina': 'angina',
        
        # Asthma related
        'asma bronkial': 'asma',
        'asma exacerbation': 'asma',
        'asma': 'asma',
        'pemburukan asma': 'asma',
        
        # Bronchitis related
        'bronkitis akut': 'bronkitis',
        'bronkitis': 'bronkitis',
        
        # Wound related
        'vulnus laceratum': 'luka robek',
        'luka robek': 'luka robek',
        'luka terbuka': 'luka robek',
        'laceration': 'luka robek',
        'vulnus excoriatum': 'luka lecet',
        'luka lecet': 'luka lecet',
        
        # Head injury related
        'cedera kepala ringan': 'ckr',
        'kepala cedera ringan': 'ckr',
        'ckr': 'ckr',
        
        # Dyspepsia related
        'dispepsia': 'dispepsia',
        'dispepsia fungsional': 'dispepsia',
        
        # Appendicitis related
        'appendisitis akut': 'appendisitis',
        'appendisitis': 'appendisitis',
        'apendisitis': 'appendisitis',
        
        # UTI related
        'infeksi saluran kemih': 'isk',
        'isk': 'isk',
        'infeksi saluran kemih akut': 'isk',
        
        # Additional mappings
        'intoleransi laktosa': 'intoleransi laktosa',
        'pneumonia': 'pneumonia',
        'hipertensi': 'hipertensi',
        'hipertensi akut': 'hipertensi',
        'hipertensi stage 1': 'hipertensi',
        'luka bakar': 'luka bakar',
        'burn injury': 'luka bakar',
        'tonsilitis': 'tonsilitis',
        'faringitis': 'faringitis',
        'vertigo': 'vertigo',
        'benign paroxysmal positional vertigo': 'vertigo',
        'bppv': 'vertigo',
        'influenza': 'influenza',
        'kolik renal': 'kolik renal',
        'batu ginjal': 'batu ginjal',
        'peritonitis': 'peritonitis'
    }
    
    # Apply mapping if available - using partial matching for more flexibility
    for key, value in mapping.items():
        if key in name:
            return value
    
    # If no mapping found, return the cleaned name
    return name

# Function to extract percentage from diagnosis text
def extract_percentage(text):
    match = re.search(r'(\d+)%', text)
    return float(match.group(1))/100 if match else 0.5  # Default to 50% if not found

In [28]:
# Function to parse diagnoses from raw answer text
def parse_diagnoses(answer_text):
    if pd.isna(answer_text):
        return []
    
    # Split by semicolon and process each diagnosis
    diagnoses = []
    percentages = []
    
    for item in answer_text.split(';'):
        item = item.strip()
        if not item:
            continue
            
        # Extract diagnosis name and percentage 
        match = re.search(r'(.*?)(?:\s+(\d+)%)?$', item)
        if match and match.group(1).strip():
            diagnoses.append(standardize_disease_name(match.group(1).strip()))
            percentages.append(extract_percentage(item))
    
    # Create a list of tuples (diagnosis, percentage)
    return list(zip(diagnoses, percentages))


In [ ]:
# Function to find the best matching answer for a given Claude answer
def select_best_answer(answers, claude_diagnoses):
    """
    Selects the best matching Qwen answer based on overlap with Claude diagnoses
    
    Args:
        answers: List of Qwen answer strings
        claude_diagnoses: List of (diagnosis, score) tuples from Claude
    
    Returns:
        best_answer: The Qwen answer with highest similarity score
    """
    if not answers or not claude_diagnoses:
        return ""
    
    best_score = -1
    best_answer = ""
    
    claude_diseases = [d[0] for d in claude_diagnoses]
    
    for ans in answers:
        parsed = parse_diagnoses(ans)
        diseases = [d[0] for d in parsed]
        print(ans, claude_diagnoses, claude_diseases, diseases)
        
        match_count = sum(1 for d in diseases if d in claude_diseases)
        match_score = match_count / max(len(claude_diseases), len(diseases), 1)

        print('match ', match_score)
        
        text_sim = calculate_cosine_similarity(
            preprocess_text(ans), 
            preprocess_text('; '.join([d for d, _ in claude_diagnoses]))
        )

        print('sim ', text_sim)
        
        combined_score = (match_score * 0.7) + (text_sim * 0.3)
        
        if combined_score > best_score:
            best_score = combined_score
            best_answer = ans
            
    return best_answer, best_score

In [30]:
# Function to calculate diagnosis matching metrics
def calculate_diagnosis_match_metrics(claude_diagnoses, diagnose):
    """
    Calculate precision, recall, and F1 score for disease matching
    
    Args:
        claude_diagnoses: List of (disease, score) tuples from Claude
        diagnose: List of (disease, score) tuples from Qwen
    
    Returns:
        dict with precision, recall, f1_score, etc.
    """
    if not claude_diagnoses or not diagnose:
        return {
            'precision': 0.0,
            'recall': 0.0,
            'f1_score': 0.0,
            'matched': 0,
            'total_ground_truth': len(claude_diagnoses),
            'total_predictions': len(diagnose)
        }
    
    # Extract just the disease names
    claude_diseases = [d[0] for d in claude_diagnoses]
    diseases = [d[0] for d in diagnose]
    
    # Count matches
    matches = sum(1 for d in diseases if d in claude_diseases)
    
    # Calculate precision: matches / qwen predictions
    precision = matches / len(diseases) if diseases else 0
    
    # Calculate recall: matches / claude diagnoses (ground truth)
    recall = matches / len(claude_diseases) if claude_diseases else 0
    
    # Calculate F1 score
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'matched': matches,
        'total_ground_truth': len(claude_diseases),
        'total_predictions': len(diseases)
    }

In [31]:
def process_medical_diagnoses():
    xlsx_path = 'evaulation/30 sample penyakit - hasil prompt LLM.xlsx'
    
    try:
        # Load data
        claude_raw = pd.read_excel(xlsx_path, sheet_name='Claude 3.5 Haiku')
        deepseek_raw = pd.read_excel(xlsx_path, sheet_name='Deepseek-V3-RAG')
        
        for df in [claude_raw, deepseek_raw]:
            df['No'] = df['No'].ffill()
            df['Question'] = df['Question'].ffill()
        
        claude_grouped = claude_raw.groupby(['No', 'Question'])
        deepseek_grouped = deepseek_raw.groupby(['No', 'Question'])

        print(deepseek_grouped)
        
        all_questions = list(set(claude_grouped.groups.keys()) | 
                             set(deepseek_grouped.groups.keys()) )
        all_questions.sort(key=lambda x: float(x[0]) if x[0] is not None and not pd.isna(x[0]) else float('inf'))

        all_results = []

        # Evaluate for each model variant
        for model_name, grouped_data in [('Deepseek', deepseek_grouped)]:
            for idx, (no, question) in enumerate(all_questions):
                try:
                    print(f"[{model_name}] Processing question {idx+1}/{len(all_questions)}: {no}")

                    if (no, question) not in claude_grouped.groups:
                        print(f"Warning: No Claude data for question {no}")
                        continue

                    # Claude data
                    claude_rows = claude_grouped.get_group((no, question))
                    claude_answers = claude_rows['Answer'].dropna().tolist()
                    claude_diagnoses = []
                    for ans in claude_answers:
                        claude_diagnoses.extend(parse_diagnoses(ans))
                    claude_explanations = claude_rows['Full Answer'].dropna().tolist()
                    claude_full_text = ' '.join(claude_explanations)

                    # Model answer
                    if (no, question) in grouped_data.groups:
                        model_rows = grouped_data.get_group((no, question))
                        model_answers = model_rows['Answer'].dropna().tolist()

                        best_model_answer, similarity_score = select_best_answer(model_answers, claude_diagnoses)
                        model_diagnoses = parse_diagnoses(best_model_answer)

                        explanation_rows = model_rows[model_rows['Answer'] == best_model_answer]
                        explanations = explanation_rows['Full Answer'].dropna().tolist()
                        model_full_text = ' '.join(explanations) if explanations else ""
                    else:
                        print(f"Warning: No {model_name} data for question {no}")
                        best_model_answer = ""
                        similarity_score = 0.0
                        model_diagnoses = []
                        model_full_text = ""

                    # Match metrics
                    diagnosis_metrics = calculate_diagnosis_match_metrics(claude_diagnoses, model_diagnoses)
                    claude_answer_text = '; '.join([f"{d} {int(p*100)}%" for d, p in claude_diagnoses])
                    
                    nlp_metrics = {
                        'answer_cosine_similarity': calculate_cosine_similarity(
                            preprocess_text(claude_answer_text), 
                            preprocess_text(best_model_answer)
                        ),
                        'full_answer_cosine_similarity': calculate_cosine_similarity(
                            preprocess_text(claude_full_text), 
                            preprocess_text(model_full_text)
                        ),
                        'bleu_score': calculate_bleu_score(
                            preprocess_text(claude_answer_text), 
                            preprocess_text(best_model_answer)
                        ),
                        'meteor_score': calculate_meteor_score(
                            preprocess_text(claude_answer_text), 
                            preprocess_text(best_model_answer)
                        ),
                        'bert_score': calculate_bert_score(
                            preprocess_text(claude_answer_text), 
                            preprocess_text(best_model_answer)
                        )
                    }

                    # Append result
                    result = {
                        'Model': model_name,
                        'No': no,
                        'Question': question,
                        'Claude_Diagnoses': claude_answer_text,
                        'Selected_Model_Answer': best_model_answer,
                        'Similarity_Score': similarity_score,
                        **diagnosis_metrics,
                        **nlp_metrics
                    }

                    all_results.append(result)

                except Exception as e:
                    print(f"Error processing question {no} for model {model_name}: {str(e)}")
                    continue

        # Final DataFrame
        results_df = pd.DataFrame(all_results)

        # Overall metrics per model
        overall_metrics = results_df.groupby('Model').agg({
            'precision': 'mean',
            'recall': 'mean',
            'f1_score': 'mean',
            'answer_cosine_similarity': 'mean',
            'full_answer_cosine_similarity': 'mean',
            'bleu_score': 'mean',
            'meteor_score': 'mean',
            'bert_score': 'mean'
        }).reset_index()
        overall_metrics['total_questions'] = results_df.groupby('Model')['No'].nunique().values

        return results_df, overall_metrics

    except Exception as e:
        print(f"Error processing Excel file: {str(e)}")
        return None, None

In [32]:
# Function to run the full evaluation and display results
print("Starting medical diagnosis evaluation...")
results_df, overall_metrics = process_medical_diagnoses()

if results_df is not None:
    results_df.to_csv('deepseek_rag_results.csv', index=False)
else:
    print("Error: Evaluation failed.")

Starting medical diagnosis evaluation...
[Deepseek] Processing question 1/50: 1.0
Demam Tifoid (Typhoid Fever) [('demam tifoid', 0.7), ('demam berdarah dengue', 0.3)] ['demam tifoid', 'demam berdarah dengue'] ['demam tifoid (typhoid fever)']
match  0.0
sim  0.41106537098251733
Malaria [('demam tifoid', 0.7), ('demam berdarah dengue', 0.3)] ['demam tifoid', 'demam berdarah dengue'] ['malaria']
match  0.0
sim  0.0
Leptospirosis [('demam tifoid', 0.7), ('demam berdarah dengue', 0.3)] ['demam tifoid', 'demam berdarah dengue'] ['leptospirosis']
match  0.0
sim  0.0
Infeksi Saluran Kemih (ISK) [('demam tifoid', 0.7), ('demam berdarah dengue', 0.3)] ['demam tifoid', 'demam berdarah dengue'] ['isk']
match  0.0
sim  0.0
Sepsis [('demam tifoid', 0.7), ('demam berdarah dengue', 0.3)] ['demam tifoid', 'demam berdarah dengue'] ['sepsis']
match  0.0
sim  0.0
[Deepseek] Processing question 2/50: 2.0
Gastroenteritis Akut [('gastroenteritis', 0.7), ('intoleransi laktosa', 0.3)] ['gastroenteritis', 'into

KeyboardInterrupt: 

In [ ]:
results_df.describe()

,No,Similarity_Score,precision,recall,f1_score,matched,total_ground_truth,total_predictions,answer_cosine_similarity,full_answer_cosine_similarity,bleu_score,meteor_score,bert_score
count,5.000000,5.0,5.0,5.0,5.0,5.0,5.000000,5.0,5.0,5.0,5.0,5.0,5.0
mean,12.400000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0
std,10.691118,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.0,0.0,0.0
min,4.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
25%,6.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0
50%,7.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0
75%,15.000000,0.0,0.0,0.0,0.0,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,0.0
max,30.000000,0.0,0.0,0.0,0.0,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,0.0
